# NLP(TUKEY)

# Luis Melendez

CARGAR TXT

In [197]:
import pandas as pd

with open('test.txt', 'r', encoding='utf-8') as f:
    lineas = f.readlines()
data = [line.strip().split(';') for line in lineas if ';' in line]
df = pd.DataFrame(data, columns=['Comentario', 'Emocion'])
df

,Comentario,Emocion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness
...,...,...
1995,i just keep feeling like someone is being unki...,anger
1996,im feeling a little cranky negative after this...,anger
1997,i feel that i am useful to my people and that ...,joy
1998,im feeling more comfortable with derby i feel ...,joy


# Funciones para normalizar

In [199]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def minusculas(texto):
    res = ""
    for x in range(len(texto)):
        if texto[x] >= "A" and texto[x] <= "Z":
            res += chr(ord(texto[x])+32)
        else:
            res += texto[x]
    return res

def quitarPuntuacion(texto):
    res = ""
    for x in range(len(texto)):
        if texto[x].isalnum() or texto[x].isspace():
            res += texto[x]
    return res

def tokens(texto):
    tokens = []
    palabra = ""
    for x in range(len(texto)):
        if texto[x] != " ":
            palabra += texto[x]
        else:
            if len(palabra) >= 1:
                tokens.append(palabra)
                palabra = ""
    if len(palabra) >= 1:
        tokens.append(palabra)
    return tokens

def eliminarStopwprds(texto):
    temp = []
    stopWords = set(stopwords.words('english'))
    for x in texto:
        if not x in stopWords:
            temp.append(x)
    return temp
            
def normalizar(texto):
    texto = minusculas(texto)
    texto = quitarPuntuacion(texto)
    texto = tokens(texto)
    texto = eliminarStopwprds(texto)
    return texto

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luigu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Normalizar

In [201]:
df["Tokens"] = df["Comentario"].apply(normalizar)
df["Comentario normalizado"] = df["Tokens"].apply(lambda x: ' '.join(x))
df

,Comentario,Emocion,Tokens,Comentario normalizado
0,im feeling rather rotten so im not very ambiti...,sadness,"[im, feeling, rather, rotten, im, ambitious, r...",im feeling rather rotten im ambitious right
1,im updating my blog because i feel shitty,sadness,"[im, updating, blog, feel, shitty]",im updating blog feel shitty
2,i never make her separate from me because i do...,sadness,"[never, make, separate, ever, want, feel, like...",never make separate ever want feel like ashamed
3,i left with my bouquet of red and yellow tulip...,joy,"[left, bouquet, red, yellow, tulips, arm, feel...",left bouquet red yellow tulips arm feeling sli...
4,i was feeling a little vain when i did this one,sadness,"[feeling, little, vain, one]",feeling little vain one
...,...,...,...,...
1995,i just keep feeling like someone is being unki...,anger,"[keep, feeling, like, someone, unkind, wrong, ...",keep feeling like someone unkind wrong think g...
1996,im feeling a little cranky negative after this...,anger,"[im, feeling, little, cranky, negative, doctor...",im feeling little cranky negative doctors appo...
1997,i feel that i am useful to my people and that ...,joy,"[feel, useful, people, gives, great, feeling, ...",feel useful people gives great feeling achieve...
1998,im feeling more comfortable with derby i feel ...,joy,"[im, feeling, comfortable, derby, feel, though...",im feeling comfortable derby feel though start...


# Representacion numerica

In [203]:
import numpy as np
import math
def obtenerVocabulario(tokens):
    voc = set()
    for x in tokens:
        voc.update(x)
    return list(voc)
    
def BoW(tokens,vocab):
    vector = [0] * len(vocab)
    for word in tokens:
        if word in vocab:
            idx = vocab.index(word)
            vector[idx] += 1
    return vector
def tf_idf(tokens, vocab):
    no_docs = len(tokens)

    idf = {}
    for word in vocab:
        n_t = sum(1 for document in tokens if word in document)
        idf[word] = math.log(no_docs / (1 + n_t))

    tf_idf_matrix = []
    for doc in tokens:
        doc_len = len(doc)
        tf_idf_vector = []
        for word in vocab:
            tf = doc.count(word) / doc_len if doc_len > 0 else 0
            tf_idf_val = tf * idf[word]
            tf_idf_vector.append(tf_idf_val)
        tf_idf_matrix.append(tf_idf_vector) 
    return np.array(tf_idf_matrix)



voc = obtenerVocabulario(df["Tokens"])
vectorBoW = []
for t in df["Tokens"]:
    vector = BoW(t, voc)
    vectorBoW.append(vector)

tokens = df["Tokens"].tolist()
tfidf_matrix = tf_idf(tokens, voc)

# Ejemplo de Representación numérica usando BoW o TF-IDF con la primer fila

In [205]:
print(tfidf_matrix[0])
print(vectorBoW[0])

[0. 0. 0. ... 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Agrupa cada emocion con una lista de palabras correspondiente

In [207]:
grupos = df.groupby("Emocion")["Tokens"].apply(lambda x: sum(x, []))
print(grupos)

Emocion
anger       [felt, anger, end, telephone, call, jest, feel...
fear        [cant, walk, shop, anywhere, feel, uncomfortab...
joy         [left, bouquet, red, yellow, tulips, arm, feel...
love        [find, odd, position, feeling, supportive, fee...
sadness     [im, feeling, rather, rotten, im, ambitious, r...
surprise    [feel, little, stunned, imagine, folks, workin...
Name: Tokens, dtype: object


# Palabras con mayor peso en cada emocion

In [209]:
listG = grupos.tolist()
vocab1 = obtenerVocabulario(listG)
vectores = [BoW(doc, vocab1) for doc in listG]
tfidf = tf_idf(listG, vocab1)
for i, emocion in enumerate(grupos.index):
    pesos = tfidf[i]
    top_indices = np.argsort(pesos)[::-1][:10]
    print(f"Palabras más representativas en '{emocion}': \n")
    print([vocab1[j] for j in top_indices])


Palabras más representativas en 'anger': 

['greedy', 'insulted', 'stubborn', 'rushed', 'impatient', 'cranky', 'irritable', 'bitter', 'bitchy', 'bothered']
Palabras más representativas en 'fear': 

['pressured', 'restless', 'shy', 'doubtful', 'afraid', 'suspicious', 'uncertain', 'intimidated', 'shaky', 'distressed']
Palabras más representativas en 'joy': 

['perfect', 'acceptable', 'virtuous', 'valued', 'determined', 'honoured', 'relaxed', 'energetic', 'pleasant', 'worthwhile']
Palabras más representativas en 'love': 

['tender', 'gentle', 'sympathetic', 'romantic', 'naughty', 'longing', 'supportive', 'delicate', 'loyal', 'supporting']
Palabras más representativas en 'sadness': 

['dull', 'embarrassed', 'lonely', 'ashamed', 'ugly', 'defeated', 'humiliated', 'unwelcome', 'homesick', 'awful']
Palabras más representativas en 'surprise': 

['strange', 'curious', 'shocked', 'handed', 'amazed', 'dazed', 'contractions', 'ludicrous', 'feng', 'age']


# Palabras compartidas

In [234]:
palabras = []

for x in vocab1:
    conteo = sum(x in grupos[emocion] for emocion in grupos.index)
    if conteo >= 2:
        palabras.append(x)

print(palabras)

['force', 'eyes', 'really', 'report', 'isolated', 'words', 'workout', 'naturally', 'studies', 'dancer', 'overwhelming', 'someone', 'gonna', 'feet', 'daily', 'crying', 'feeling', 'area', 'elevator', 'talks', 'drive', 'photo', 'knew', 'brothers', 'little', 'definately', 'fond', 'routine', 'beautiful', 'leave', 'sorts', 'hold', 'follow', 'alot', 'sent', 'together', 'sized', 'posting', 'control', 'finished', 'town', 'shoes', 'dont', 'front', 'toward', 'allowing', 'apprehensive', 'guess', 'breaking', 'deeply', 'desperate', 'credit', 'familiar', 'effort', 'head', 'willing', 'cycle', 'upon', 'fact', 'happier', 'throughout', 'ability', 'also', 'text', 'urge', 'skin', 'black', 'phone', 'mood', 'accommodate', 'secret', 'making', 'god', 'water', 'spent', 'classic', 'think', 'joy', 'due', 'two', 'amazed', 'theater', 'meaning', 'wear', 'ad', 'red', 'try', 'opinions', 'strange', 'comfortable', 'began', 'married', 'plot', 'world', 'late', 'pick', 'twice', 'injury', 'fresh', 'capable', 'going', 'well'

# Palabras con diferente significado  
Si la misma palabra tiene diferente peso pero aparece en varias emociones se considera que tienen diferente significado

In [238]:
important_words = {}
min_tf_idf = 0.01           
relative_difference = 2     

emociones = list(grupos.index) 

for i in range(len(vocab1)):  
    palabra = vocab1[i]
    pesos = [tfidf[e][i] for e in range(len(emociones))]  
    max_valor = max(pesos)
    idx_max = pesos.index(max_valor)
    emocion_max = emociones[idx_max]

    if max_valor < min_tf_idf:
        continue  

    suma_otros = sum(pesos) - max_valor
    promedio_otros = suma_otros / (len(pesos) - 1) if len(pesos) > 1 else 0

    if promedio_otros > 0 and (max_valor / promedio_otros >= relative_difference):
        important_words[palabra] = emocion_max


for palabra, emocion in important_words.items():
    print(f"'{palabra} en  '{emocion}")

'strange en  'surprise
'curious en  'surprise


# Palabras mas frecuentes

In [256]:
tokensT = df["Tokens"].tolist()

frecuencia = {}
for x in tokensT:
    for y in x:
        if y in frecuencia:
            frecuencia[y] += 1
        else:
            frecuencia[y] = 1


ordenadas = sorted(frecuencia.items(), key=lambda x: -x[1])

print("\nTop 15 palabras más frecuentes:")
for i in range(15):
    palabra, freq = ordenadas[i]
    print(f"{palabra}: {freq} veces")



Top 15 palabras más frecuentes:
feel: 1394 veces
feeling: 646 veces
like: 373 veces
im: 310 veces
really: 118 veces
know: 102 veces
time: 100 veces
get: 97 veces
people: 96 veces
little: 95 veces
one: 84 veces
even: 79 veces
would: 75 veces
ive: 73 veces
want: 72 veces


# longitud promedio de las oraciones

In [274]:
emociones = df["Emocion"].tolist()
tokens_por_oracion = df["Tokens"].tolist()

sumaL = {}
conteo = {}


for i in range(len(tokens_por_oracion)):
    emocion = emociones[i]
    longitud = len(tokens_por_oracion[i])

    if emocion in sumaL:
        sumaL[emocion] += longitud
        conteo[emocion] += 1
    else:
        sumaL[emocion] = longitud
        conteo[emocion] = 1

for emocion in sumaL:
    promedio = sumaL[emocion] / conteo[emocion]
    print(f"{emocion}: {promedio:.2f} palabras por oración")


sadness: 9.26 palabras por oración
joy: 9.27 palabras por oración
fear: 8.86 palabras por oración
anger: 9.55 palabras por oración
love: 9.74 palabras por oración
surprise: 9.39 palabras por oración


# Palabras exclusivas de cada clase

In [252]:
emociones = list(grupos.index)
palabras_por_emocion = {}

for x in emociones:
    palabras = grupos[x]  
    palabras_por_emocion[x] = []
    for y in palabras:
        if y not in palabras_por_emocion[x]:
            palabras_por_emocion[x].append(y)

for x in emociones:
    propias = palabras_por_emocion[x]
    otras = []
    for y in emociones:
        if y != x:
            otras += palabras_por_emocion[y]

  
    exclusivas = []
    for x in propias:
        if x not in otras:
            exclusivas.append(x)

    print(f"\n{emocion} ({len(exclusivas)} palabras): {exclusivas}") 


surprise (431 palabras): ['telephone', 'jest', 'pre', 'menstrual', 'walrus', 'bcoz', 'n', 'publicity', 'outraged', 'recommend', 'visiting', 'crowded', 'farmers', 'irish', 'democratic', 'enact', 'electricity', 'garbage', 'envious', 'frog', 'neck', 'thoughtful', 'soften', 'grab', 'smooth', 'frame', 'pals', 'offhand', 'remark', 'crumble', 'fucked', 'piddled', 'weather', 'kidding', 'impatient', 'bitter', 'ellie', 'insulted', 'douche', 'baggy', 'lothaire', 'according', 'insults', 'somebody', 'quinn', 'barney', 'ted', 'robin', 'flashed', 'hockey', 'degree', 'coerced', 'rushed', 'flustered', 'pool', 'debate', 'dichotomy', 'irritable', 'unresolved', 'severe', 'effect', 'cashier', 'fumble', 'coins', 'purse', 'lies', 'shattered', 'delivery', 'justice', 'conceptualized', 'courts', 'emphasis', 'stella', 'noticing', 'outsourcing', 'picky', 'sod', 'dissatisfied', 'outsource', 'struggled', 'mightily', 'cranky', 'eleven', 'thirty', 'dhawan', 'tweet', 'javascript', 'platform', 'royally', 'wreaking', '

# Concluciones

anger: 
Las palabras con mayor peso fueron:
['greedy', 'insulted', 'stubborn', 'rushed', 'impatient', 'cranky', 'irritable', 'bitter', 'bitchy', 'bothered']
Describen las cualidades de una persona frustrada, podemos decir que los comentarios etiquetados como anger suelen estar relacionados con la frustracion

fear: 
Las palabras con mayor peso fueron:
['pressured', 'restless', 'shy', 'doubtful', 'afraid', 'suspicious', 'uncertain', 'intimidated', 'shaky', 'distressed']
Describen a una persona con baja autoestima, o personas intimidadas

joy: 
Las palabras con mayor peso fueron:
['perfect', 'acceptable', 'virtuous', 'valued', 'determined', 'honoured', 'relaxed', 'energetic', 'pleasant', 'worthwhile']
Las palabras parecen ser cualidades de algun producto que es aceptado por las personas, son cualidades de una persona que puede alcanzar el exito facilmente

love: 
Las palabras con mayor peso fueron:
['tender', 'gentle', 'sympathetic', 'romantic', 'naughty', 'longing', 'supportive', 'delicate', 'loyal', 'supporting']
Las palabras parecen ser cualidades que se busca en una persona cuando se trata de relacionarse de manera amorosa o amistosa

sadness: 
Las palabras con mayor peso fueron:
['dull', 'embarrassed', 'lonely', 'ashamed', 'ugly', 'defeated', 'humiliated', 'unwelcome', 'homesick', 'awful']
Parecen describir algun tipo de fracaso o cualidades que son negativas 

surprise: 
Las palabras con mayor peso fueron:
['strange', 'curious', 'shocked', 'handed', 'amazed', 'dazed', 'contractions', 'ludicrous', 'feng', 'age']
Las palabras tienen que ver con algo nuevo y parece ser que es algo asobroso o que deja aturdido a las personas